# Группа: DST-48
**Sergey Pinaev - binom1982@gmail.com**
# Финальный проект 
Предсказание стоимости домов, основываясь на истории предложений.  

Ссылка на соревнование: [[SF-DST] Car Price prediction](https://www.kaggle.com/c/sf-dst-car-price-prediction-part2)

<p align="center" width="100%">
<img src="https://habrastorage.org/r/w1560/getpro/habr/upload_files/080/2a6/5e7/0802a65e78ee2bd84388c0d1ebab98d5.png" width="auto"/>
</p>
<hr>

## Task

Ко мне обратился представитель крупного агентства недвижимости со следующей проблемой:

Мои риелторы тратят катастрофически много времени на сортировку объявлений и поиск выгодных предложений. Поэтому их скорость реакции, да и, сказать по правде, качество анализа не дотягивает до уровня конкурентов. А это сказывается на наших финансовых показателях. Твоя задача — разработать модель, которая бы позволила обойти конкурентов по скорости и качеству совершения сделок. Датасет прикладываю.

**Цель**: разработать сервис, который будет предсказывать стоимость домов, основываясь на истории предложений.

## Data Description:

В датасете 18 колонок

Целевая переменная **target** - цена объекта недвижимости
* **status** - статус объявления
* **private pool** - наличие бассейна, вероятно дублируется столбцом **PrivatePool**
* **propertyType** - тип объекта недвижимости
* **street**- адрес
* **baths** - количество ванных комнат
* **homeFacts**- информация о доме. Этот столбец надо парсить
* **fireplace** - наличие камина
* **city** - город
* **schools** - рейтинг и близость образовательных учреждений. Нужно парсить
* **sqft** - площадь в квадратных футах
* **zipcode** - почтовый индекс
* **beds**- количество спальен (или количество кроватей) 
* **state** - штат
* **stories** - вероятно, количество владельцев или продаж этого дома
* **mls-id** - идентификационный номер в системе MLS. Вероятно, дополняется столбцом **MlsId**


## Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import random
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import sys
import PIL
import cv2
import re
import json

# import datetime, time
from datetime import timedelta, datetime, date, time

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import StackingRegressor
from catboost import CatBoostRegressor

from sklearn.linear_model import LinearRegression, LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.feature_selection import RFE # Рекурсивное устранение признаков (RFE)
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error

# # keras
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import albumentations

# plt
import matplotlib.pyplot as plt
import seaborn as sns
#увеличим дефолтный размер графиков
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
#графики в svg выглядят более четкими
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline
plt.style.use('fivethirtyeight')
# Подключаем форматирование Markdown
from IPython.display import Markdown

print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)
print('Tensorflow   :', tf.__version__)
print('Keras        :', tf.keras.__version__)

# Чтобы вычисления проходили на GPU необходимо чтобы tensorflow определил GPU.
# Как это сделать практически без боли написано здесь https://artificialintelligence.so/forums/discussion/how-to-install-tensorflow/
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices()
# Подключим видеокарту
!nvidia-smi -L
# !pip freeze > requirements.txt

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
isWorking = False # флаг для запуска кода

## Functions

In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))


def get_other_values(series, percentile=90):
    '''
    Возвращает список значений ниже определенного порога, которые необходимо переменовать
    '''
    value_counts = series.value_counts()
    # Найдем значения описывающие percentile всех значений
    top_values = int(np.percentile(series.value_counts(dropna=False), percentile))
    # Оставим только percentile значений
    other = value_counts[value_counts < top_values].index
#     display(other)
    return other

def get_emission_limits(data_series):
    '''
    Возвращает кортеж границ выбросов вычесленных на основе межвартильного размаха и 1 и 99 перцентиль.
    При необходимости данные выходящие за нижнюю и верхнюю границу можно не удалять а присваивать им значение  1 и 99 перцентиля.
    df.loc[df['column'] < min_emission_limits, 'column'] = perc01
    df.loc[df['column'] > max_emission_limits, 'column'] = perc99
    '''
    perc25, perc75 = np.percentile(data_series,[25,75])
    perc1, perc99 = np.percentile(data_series,[1,99])
    # perc0, perc100 = np.percentile(data_series,[0,100])
    # print(perc0, perc100) # Иногда выбросы меньше/больше min/max значений так что можно было бы доработать
    
    IQR = perc75 - perc25
    min_emission_limits = perc25 - 1.5*IQR
    max_emission_limits = perc75 + 1.5*IQR

    '''Выводит распределение данных для каждого столбца в датасете'''
    l = ['| P1 | Min emission | P25 | IQR | P75 | Max emission | P99 |',
         '|-----|:-----:|----|-----|----|:-----:|-----|',
        f'|{perc1}|{min_emission_limits}|{perc25}|{IQR}|{perc75}|{max_emission_limits}|{perc99}|']
            
    display(Markdown('\n'.join(l)))
    return perc1, min_emission_limits, max_emission_limits, perc99


def get_boxplot(dataset, column, value, axes, kind=True):
    '''Отображет boxplot'''
    #fig, axes = plt.subplots(figsize=(10, 4))
    
    if kind:
        sns.boxplot(x=column, y=value, data=dataset, ax=axes)
    else:
        sns.violinplot(x=column, y=value, data=dataset, ax=axes)
    # plt.xticks(rotation=45)
    # ax.set_title('Boxplot for ' + column)
    # plt.show()
    
    
def myround(x, prec=2, base=.05):
    return round(base * round(float(x)/base), prec)



## Loading data

In [ ]:
PATH = "G:/Datasets/finale_project/data.csv/data.csv"
data = pd.read_csv(PATH)

In [ ]:
data.columns = data.columns.map(lambda x: x.replace(' ', '_').replace('-', '_'))
display(data.head(4).T)

In [ ]:
data.info()

In [ ]:
data.describe()

## EDA

In [ ]:
# список столбцов для удаления
delete_col = []

### Columns "mls_id" and "MlsId"

In [ ]:
# Начнем с удаления дубликатов в столбцах в названиях которых присутвует "id". 
print('Пропуски:', data.mls_id.isna().sum())
print('Уникальных:', data.target.nunique())

MlsId =  data.MlsId.value_counts(dropna=False)
MlsId[MlsId >= 2]

In [ ]:
display(data[data.MlsId == '58868465'])

По самому частому 'MlsId' 6 записей по 3 в SAN ANTONIO и SEATTLE.
Данные дублируют друг друга. Желательно удалить строки дубликатов.

В них больше всего пропусков, соответственно, поэтому дополнение датасета данными из другой базы данных, 
например американской базы риелторов по этим id, представляется нецелесообразным, 
так как датасет будет дополнен неравномерно.

In [ ]:
# TODO Пока удалим столбец (по этому столбцу позже можно удалить дубли)
delete_col.append("MlsId")
delete_col.append("mls_id")


In [ ]:
# # Визуализируем пропуски:
# fig, ax = plt.subplots(figsize=(15,10))
# sns_heatmap=sns.heatmap(data.isnull(), yticklabels=False, cbar=False, cmap= 'coolwarm')

### Column "status"

In [ ]:
print('Пропуски:', data.status.isna().sum())
print('Уникальных:', data.status.nunique())

data.status = data.status.str.upper()
display(data.status.value_counts(dropna=False)[:20])

Больше 250.000 записей о продаже готовой недвижимости **ACTIVE**, **FOR SALE**,
**NaN** значений около 40.000.

Много записей о :
* строящемся жилье **PENDING** 
* о выкупе у должников **FORECLOUSER**
* отдельно указываются новостройки **NEW CONSTRUCTION**.
 
Не очень понятна принципиальная разница между категориями 'ACTIVE' и 'FOR SALE'. Они по сути отражают одно и то же сотояние объекта недвижимоти - готовность объекта к продаже(активное объявление). 

Насчет заполнения пропусков. Раз они присутствуют в датасете - они должны быть активными и готовыми к продаже, как и абсолютное большинство других объявлений.

In [ ]:
def parse_status(value):
    if type(value)==str:
        s = str(value)
        if 'AUCTION' in s: return 'AUCTION'
        if 'COMIN' in s or 'DILIGE' in s: return 'COMING'
        if 'RENT' in s: return 'RENT'
        if 'LEASE' in s: return 'RENT'
        if 'PURCH' in s: return 'RENT'
        if 'PEND' in s: return 'PENDING'
        if 'FOREC' in s: return 'FORECLOSURE'
        if 'NEW' in s: return 'NEW'
        if 'CONTRACT' in s: return 'CONTRACT'
        if 'CONTINGE' in s: return 'CONTINGENT'
        if 'SOLD' in s or 'CLOSED' in s or 'ACCEPT' in s: return 'SOLD'
        if 'ACTIV' in s: return 'ACTIVE'
        if 'FOR SALE' in s: return 'FOR SALE'
        if 'BACKUP' in s: return 'CONTRACT'
        if 'BACK' in s or 'EXTEND' in s: return 'ACTIVE'
        if 'CONTINUE' in s: return 'ACTIVE'
        if s == 'C' or  s == 'CT': return 'CONTRACT'
        if s == 'P': return 'PENDING'
        if s == 'PS' or s == 'PF' or s == 'PI': return 'PENDING'
       
        else: return s
    else:
         # if s == 'nan' : return  "NO_DATA" # 'ACTIVE'
        return "NO_DATA"

In [ ]:
status = data.status.map(parse_status)
display(status.value_counts(dropna=False))
data.status = status.astype("category")
# data.status.hist(figsize=(15,5), log=False, bins=50, xrot=90);
# plt.tight_layout()

In [ ]:
# Видно, что для объявления о сдаче недвижимости в аренду, цена считается за месяц и в стоимости присутствует
# приписка о помесячной оплате '/mo'. 
# Наш датасет должен содеражть данные о продаже жилья, а сдача жилья это уже для другой задачи. 
# Поэтому удалим удалим 424 записи со статусом 'RENT'.
data[data.status == 'RENT']['target'][:8]

# Удаляем записи
data = data[data.status != 'RENT']

### Columns "private pool" and "PrivatePool"

In [ ]:
# private_pool
print('Пропуски:', data.private_pool.isna().sum())
print('Уникальных:', data.private_pool.nunique())
data.private_pool = data.private_pool.str.upper()

display(data.private_pool.value_counts(dropna=False))
data.private_pool.fillna(0, inplace=True)
data.private_pool = data.private_pool.apply(lambda x: 1 if x == 'YES' else 0).astype('bool')
display(data.private_pool.value_counts(dropna=False))

# Данные о бассеинах не пересекаются

In [ ]:
# PrivatePool
print('Пропуски:', data.PrivatePool.isna().sum())
print('Уникальных:', data.PrivatePool.nunique())
data.PrivatePool = data.PrivatePool.str.upper()

display(data.PrivatePool.value_counts(dropna=False))
# data.PrivatePool = data.PrivatePool.apply(lambda x: x if type(x)!=str else x.lower())
data.PrivatePool.fillna(0, inplace=True)
data.PrivatePool = data.PrivatePool.apply(lambda x: 1 if x == 'YES' else 0).astype('bool')
display(data.PrivatePool.value_counts(dropna=False))

Данные из столбцов не пересекаются.
Заполним NaN нулями, заменим 'Yes' и 'yes' на единицы и объединим эти два столбца

In [ ]:
# TODO Я пока не могу понять почему эти данные не пересекаются, тут чтото не так!
data['private_pool_union'] = False
data.loc[data['private_pool'], 'private_pool_union'] = True
data.loc[data['PrivatePool'], 'private_pool_union'] = True
display(data.private_pool_union.value_counts(dropna=False))

In [ ]:
# TODO Пока удалим столбец (по этому столбцу позже можно удалить дубли)
delete_col.append('private_pool')
delete_col.append('PrivatePool')

### Column "propertyType"

In [ ]:
print('Пропуски:', data.propertyType.isna().sum())
print('Уникальных:', data.propertyType.nunique())
data.propertyType = data.propertyType.str.upper()
display(data.propertyType.value_counts(dropna=False))

# propertyType = data.propertyType.apply(lambda x: x if type(x)!=str else x.replace(' ', '').replace('-', ''))
# propertyType = propertyType.apply(lambda x: x if type(x)!=str else x.replace('singlefamilyhome', 'singlefamily'))

# display(propertyType.value_counts(dropna=False))
# Имеем 1280 описаний типа недвижимости. Надо разбить на разумное число категорий.

In [ ]:
def parse_property_type(s):
    s = str(s)
    if 'SINGLE' in s or 'TRADITIONAL' in s: return 'SINGLE'
    if 'CONDO' in s or 'FLAT' in s: return 'FLAT'
    if 'TOWNH' in s: return 'TOWNHOUSE'
    if 'COOP' in s or 'CO-OP' in s: return 'COOP'
    if 'LAND' in s: return 'LAND'
    if 'MULTI' in s: return 'MULTI'
    if 'CONTEMPO' in s: return 'CONTEMPORARY'
    if 'MOBI' in s or 'CARRI' in s: return 'MOBILE'
    if 'TWO STOR' in s or '2 STOR' in s: return 'TWO-STORY'
    if 'ONE STOR' in s or '1 STOR' in s: return 'ONE-STORY'
    if 'STOR' in s in s: return 'MULTY-STORY'
    if 'DETA' in s or 'DETA' in s: return 'DETACHED'
    # if 'MID' in s or '5-9' in s  or '4' in s or '3' in s: return 'MID-RISE'
    # if 'LOW' in s: return 'LOW-RISE'
    # if 'HIGH' in s or 'UNIT' in s: return 'HIGH-RISE'
    if 'RISE' in s or 'UNIT' in s or 'HIGH' in s: return 'FLAT'
    if 'PENT' in s: return 'PENTHOUSE'
    if 'RAN' in s: return 'RANCH'
    if 'GARD' in s: return 'GARDEN HOME'
    if 'CUST' in s or 'MANUF' in s or 'CRAFT' in s: return 'CUSTOM'
    if 'ATTA' in s or 'PLEX' in s: return 'ATTACHED'
    if 'FARM' in s: return 'FARM'
    if 'LEVEL' in s: return 'SPLIT-LEVEL'
    if 'OTHER' in s: return 'OTHER'
    if 'COLO' in s: return 'COLONIAL'
    if 'WARE' in s or 'COM' in s: return 'COMMERCICAL'
    if 'COTT' in s or 'RESID' in s or 'COURT' in s: return 'COTTAGE'
    if 'BOAT' in s: return 'BOATHOUSE'

    if s == 'nan' or s == '': 
        return 'SINGLE'

    else:
         return 'OTHER'

In [ ]:
data.propertyType = data.propertyType = data.propertyType.map(parse_property_type).astype('category')
display(data.propertyType.value_counts(dropna=False))

# data.propertyType.hist(figsize=(15,5), log=False, bins=200, xrot=90, orientation="vertical");
# plt.tight_layout()

In [ ]:
#mask = (data.propertyType != 'FARM') #and (data.propertyType != 'COMMERCICAL') and (data.propertyType != 'BOATHOUSE')
mask = ~data.propertyType.isin(['FARM', 'COMMERCICAL', 'BOATHOUSE'])
data = data[mask]
display(data.propertyType.value_counts(dropna=False))
# propertyType.propertyType[~mask]

### Column "street"

In [ ]:
print('Пропуски:', data.street.isna().sum())
print('Уникальных:', data.street.nunique())
data.street = data.street.str.upper()
display(data.street.value_counts(dropna=False))

Более 336 тысяч уникальных адресов. 
Возможно можно было бы извлечь из адресов геолокацию, для определения новых признаков таких как район, центр и тд. и посчитать для них среднюю или медианную цену. Но пока времени на это нет, если останется то попробую.

Почти в каждом адресе есть аббревиатуры типа улицы, например 'BLVD' -бульвар, 'ROAD' - дорога и т.п. а также номер дома.

Попробую из столбца 'street' сделать категориальный признак с типа улицы.   

In [ ]:
def parse_street(s):
    s = str(s)
    if 'ADDRESS' in s: return 'NO_ADDRESS'
    elif 'BLVD' in s or 'BOULEVARD' in s: return 'BOULEVARD'
    elif 'WAY' in s: return 'HWAY'
    elif 'CIR' in s: return 'CIRCLE'
    elif 'CT' in s or 'COURT' in s: return 'COURT'
    elif 'DR' in s or 'DRIVE' in s: return 'DRIVE'
    elif 'RD' in s or 'ROAD' in s: return 'ROAD'
    elif 'AVE' in s: return 'AVENUE'
    elif 'ST' in s or 'STREET' in s: return 'STREET'
    elif 'PL' in s: return 'PLACE'
    elif 'LANE' in s: return 'LANE'
    elif 'TR' in s or 'TRL' in s or 'TRAIL' in s: return 'TRAIL'
    elif 'PARK' in s: return 'PARK'
    else: return 'OTHER'

In [ ]:
data['street_type'] = data.street.map(parse_street).astype('category')
# TODO: Позже можно попробовать добавить новые признаки с средней/медианной ценой

display(data.street_type.value_counts(dropna=False))
data.street_type.hist(figsize=(12,5), log=False, bins=50, xrot=90);
plt.tight_layout()

# Удалим street
delete_col.append('street')

In [ ]:
# plt.figure(figsize=(12,5))
# ax = sns.boxplot(x="street_type", y="target_log", data=data);
# ax.set_xticklabels(ax.get_xticklabels(),rotation=90);

In [ ]:
# Создадим признак наличия адреса
data['has_address'] =  data.street_type.apply(lambda x: False if x == 'NO_ADDRESS' else True).astype(bool)
display(data.has_address.value_counts(dropna=False))

# Признак has_address практически не влияет на цену, но пока оставим
# plt.figure(figsize=(12,5))
# ax = sns.boxplot(x="has_address", y="target_log", data=data);
# ax.set_xticklabels(ax.get_xticklabels(),rotation=90);

In [ ]:
delete_col.append('street')

### Column "baths"

In [ ]:
print('Пропуски:', data.baths.isna().sum())
print('Уникальных:', data.baths.nunique())
data.baths = data.baths.str.upper()
display(data.baths.value_counts(dropna=False))

У более чем 100000 записей нет информации о наличии ванной комнаты (TODO нужно подумать как быть)!
Оказывается, что бывают ванны (https://www.realtor.com/advice/buy/what-is-a-half-bath/):
* 1.0 полноценная - должна содержать четыре основных приспособления: унитаз, раковину, ванну и душ (или комбинацию душа и ванны).
* 0.5 полуванна - имеет только два из четырех основных компонентов ванной комнаты, обычно унитаз и раковину
* 0.75 трехчетвертная ванна - в ней не хватает одного из четырех перечисленных выше приспособлений. Чаще всего это будет ванна.
* 0.25 четверть ванна - комната только с одним из четырех элементов, обычно это туалет

Если в доме указано, что в нем три ванные комнаты и две полуванные, почему бы просто не сложить их все вместе и не сказать, что ванных комнат четыре? Это казалось бы логичным, но каждая ванная комната должна быть указана отдельно, потому что это дает покупателям жилья лучшее представление об особенностях дома и их возможностях, когда им просто нужно уйти.

In [ ]:
def parse_baths(s):
    s = str(s)
    if s == '0': return 0.0
    if s == '': return 1.0
    if s == 'nan': return 1.0
    if 'SQ. FT.' in s: return 1.0
    if '-' in s or '—' in s or '~' in s: return 0.0
    if 'SEMIMOD' in s: return 1.0
    #if '/' in s: print(s)

    s = s.replace(',', '.')
    f = re.findall(r'\d*\.\d+|\d+', s)
    try:
        n = float(f[0])
        if n >= 200:
            n = n / 1000.
        return n
    except:
        print(s)
  
    return 1.0

In [ ]:
baths = data.baths.map(parse_baths)#.astype('category')
# TODO: Нужно дополнительно обработать!

display(baths.value_counts(dropna=False))
data.baths = baths.round(1)
display(data.baths.value_counts(dropna=False))
# data.baths.hist(figsize=(12,5), log=False, bins=50, xrot=90);
# plt.tight_layout()

In [ ]:
perc1, min_emission_limits, max_emission_limits, perc99 = get_emission_limits(data.baths)

data.baths.loc[data.baths < min_emission_limits] = 0 #int(perc1)
data.baths.loc[data.baths > max_emission_limits] = int(perc99)

In [ ]:
# Заменим выбросы минимальным значением 0 и максимальным 99 перцентилем
perc1, min_emission_limits, max_emission_limits, perc99 = get_emission_limits(data.baths)
display(data.baths.value_counts(dropna=False))

In [ ]:
data.baths.hist(figsize=(12,5), log=False, bins=50, xrot=90);
plt.tight_layout()
# data.info()

### Column "homeFacts"

In [ ]:
print('Пропуски:', data.homeFacts.isna().sum())
print('Уникальных:', data.homeFacts.nunique())
# display(data.homeFacts.value_counts(dropna=False))
data.homeFacts[0]

In [ ]:
def clen(str):
  str = str.replace("''", '0')
  str = str.replace("'", "")
  str = str.replace("None", '0')
  str = str.replace("No Data", '0')
  str = str.replace("No Info", '0')
  str = str.replace("/sqft", '')
  str = str.replace("$", '')
  return str

In [ ]:
def parse_home_facts(s):
  res = []
  s = s.split("{'atAGlanceFacts': [{'factValue': ")[1]
  s = s.split(", 'factLabel': 'Year built'}, {'factValue': ")
  s[0] = clen(s[0])
  # year = s[0]
  res.append(s[0])
  s = s[1].split(", 'factLabel': 'Remodeled year'}, {'factValue': ")
  s[0] = clen(s[0])
  # remo = s[0]
  res.append(s[0])
  s = s[1].split(", 'factLabel': 'Heating'}, {'factValue': ")
  # heat = s[0]
  res.append(s[0])
  s = s[1].split(", 'factLabel': 'Cooling'}, {'factValue': ")
  # cool = s[0]
  res.append(s[0])
  s = s[1].split(", 'factLabel': 'Parking'}, {'factValue': ")
  s[0] = clen(s[0])
  # park = s[0]
  res.append(s[0])
  s = s[1].split(", 'factLabel': 'lotsize'}, {'factValue': ")
  # size = s[0]
  res.append(s[0])
  s = s[1].split(", 'factLabel': 'Price/sqft'}]}")
  s[0] = clen(s[0])
  # price = s[0]
  res.append(s[0])
  return int(res[0]), int(res[1]), res[2], res[3], res[4], res[5], res[6]

In [ ]:
homeFacts = data.homeFacts.map(parse_home_facts)

In [ ]:
new_columns = ['hf_built_year', 'hf_remodeled_year' , 'hf_heating', 'hf_cooling', 'hf_parking', 'hf_lotsize' , 'hf_price_sqft' ]
hf_data = pd.DataFrame.from_records(homeFacts, columns = new_columns)
display(hf_data.head(5))
# display(hf_data.info())

### Dataset analysis "hf_data"

In [ ]:
# hf_data.hf_built_year
print('Пропуски:', hf_data.hf_built_year.isna().sum())
print('Уникальных:', hf_data.hf_built_year.nunique())
display(hf_data.hf_built_year.value_counts(dropna=False))
hf_data.hf_built_year.describe()
# hf_data.hf_built_year.value_counts(dropna=False).hist(figsize=(12,5), log=False, bins=50, xrot=0);
# plt.tight_layout()

In [ ]:
# Почистим выбросы 
get_emission_limits(hf_data.hf_built_year)
# data.enginePower.loc[data.enginePower < min_emission_limits] = int(perc1)
# data.enginePower.loc[data.enginePower > max_emission_limits] = int(perc99)
hf_data.hf_built_year[(hf_data.hf_built_year < 1700) | (hf_data.hf_built_year > 2025)].value_counts(dropna=False)

In [ ]:
hf_data.hf_built_year.replace(559990649990, 0, inplace=True)
hf_data.hf_built_year.replace(1, 0, inplace=True)
hf_data.hf_built_year.replace(1208, 0, inplace=True)
hf_data.hf_built_year.replace(1057, 0, inplace=True)
hf_data.hf_built_year.replace(1060, 0, inplace=True)
hf_data.hf_built_year.replace(1019, 0, inplace=True)

In [ ]:
# hf_remodeled_year
print('Пропуски:', hf_data.hf_remodeled_year.isna().sum())
print('Уникальных:', hf_data.hf_remodeled_year.nunique())
display(hf_data.hf_remodeled_year.value_counts(dropna=False))
hf_data.hf_remodeled_year.describe()
# hf_data.hf_remodeled_year.value_counts(dropna=False).hist(figsize=(12,5), log=False, bins=50, xrot=0);
# plt.tight_layout()

In [ ]:
# Почистим выбросы, год реконструкии должен быть больше или равен году реконструкции и меньше 2022 года, если 0 то не ремонтировался
get_emission_limits(hf_data.hf_remodeled_year)

hf_data[((hf_data.hf_remodeled_year < hf_data.hf_built_year) | (hf_data.hf_remodeled_year > 2025)) & hf_data.hf_remodeled_year != 0].head(5) #.value_counts(dropna=False)

In [ ]:
# У 1481 дома, реконструкция проводилась раньше постройки, что невозможно! Присвоим таким значениям 0
hf_data.loc[((hf_data.hf_remodeled_year < hf_data.hf_built_year) | (hf_data.hf_remodeled_year > 2025)) & hf_data.hf_remodeled_year != 0] = 0
hf_data[((hf_data.hf_remodeled_year < hf_data.hf_built_year) | (hf_data.hf_remodeled_year > 2025)) & hf_data.hf_remodeled_year != 0].head(5)

In [ ]:
# Создадим признак реконструкции и признак числа лет прошедших после постройки
hf_data['hf_has_remodeled'] = hf_data.hf_built_year != 0
display(hf_data.hf_has_remodeled.value_counts(dropna=False))

In [ ]:
# Создадим признак: количество лет между строительством и модернизацией, если ремонта небыло, то поставим -1
hf_data['hf_years_before_remodeled'] = (hf_data.hf_remodeled_year - hf_data.hf_built_year)
hf_data.hf_years_before_remodeled = hf_data.hf_years_before_remodeled.apply(lambda p: p if p >= 0 else -1 )
display(hf_data.hf_years_before_remodeled.value_counts(dropna=False))
# TODO Признак не идеальный у него много выбросов, нужно подумать...
get_emission_limits(hf_data.hf_years_before_remodeled)

In [ ]:
# hf_heating
print('Пропуски:', hf_data.hf_heating.isna().sum())
print('Уникальных:', hf_data.hf_heating.nunique())
hf_data.hf_heating = hf_data.hf_heating.str.upper()
display(hf_data.hf_heating.value_counts(dropna=False)[:20])
# hf_data.hf_heating.describe()

In [ ]:
def parse_heat(s):
    s = str(s)
    if len(s) == 0 or s == '' or s =="''" or s == ' ' or 'NO DATA' in s: return 'NO_DATA'
    if 'AIR' in s or 'HEAT PUMP' in s: return 'AIR'
    if 'GAS' in s or 'PROPANE' in s: return 'GAS'
    if 'ELECTRIC' in s: return 'ELECTRIC'
    if 'NONE' in s or 'NO' in s: return 'NONE'
    if 'CENTRAL' in s: return 'CENTRAL'
    else: return 'OTHER'

In [ ]:
hf_data.hf_heating = hf_data.hf_heating.map(parse_heat).astype('category')
display(hf_data.hf_heating.value_counts(dropna=False))

In [ ]:
# hf_cooling
print('Пропуски:', hf_data.hf_cooling.isna().sum())
print('Уникальных:', hf_data.hf_cooling.nunique())
hf_data.hf_cooling = hf_data.hf_cooling.str.upper()
display(hf_data.hf_cooling.value_counts(dropna=False)[:20])

In [ ]:
def parse_cooling(s):
    s = str(s)
    if len(s) == 0 or s == '' or s =="''" or s == ' ' or 'NO DATA' in s: return 'NO_DATA'
    #if 'EVAPORATIVE' in s: return 'EVAPORATIVE'
    if 'CENTRAL' in s: return 'CENTRAL'
    if 'NONE' in s or 'NO' in s: return 'NONE'
    else: return 'OTHER'

In [ ]:
hf_data.hf_cooling = hf_data.hf_cooling.map(parse_cooling).astype('category')
display(hf_data.hf_cooling.value_counts(dropna=False))

In [ ]:
# hf_parking
print('Пропуски:', hf_data.hf_parking.isna().sum())
print('Уникальных:', hf_data.hf_parking.nunique())
hf_data.hf_parking = hf_data.hf_parking.str.upper()
display(hf_data.hf_parking.value_counts(dropna=False)[:20])

In [ ]:
def parse_parking(s):
    s = str(s)
    #if len(s) == 0 or s == '' or s =="''" or s == ' ' or 'NO DATA' in s: return 'NO_DATA'
    if '1' in s or "ONE" in s or 'SING' in s: return 'ONE'
    elif '2' in s or "TWO" in s or "DOUB" in s: return 'TWO'
    elif '3' in s: return 'THREE'
    elif 'ATTACHED' in s: return 'ATTACHED'
    elif 'DETACHED' in s: return 'DETACHED'
    elif 'OFF' in s: return 'OFF_STREET'
    elif 'ON' in s: return 'ON_STREET'
    elif 'CAR' in s: return 'CARPORT'
    elif '4' in s or '5' in s or '6' in s or '7' in s or '8' in s or '9' in s: return 'FOUR_OR_MORE'
    elif 'NONE' in s or 'NO' in s or s == '0': return 'NONE'
    else: return 'OTHER'

In [ ]:
hf_data.hf_parking = hf_data.hf_parking.map(parse_parking).astype('category')
display(hf_data.hf_parking.value_counts(dropna=False))

In [ ]:
# hf_lotsize
print('Пропуски:', hf_data.hf_lotsize.isna().sum())
print('Уникальных:', hf_data.hf_lotsize.nunique())
hf_data.hf_lotsize = hf_data.hf_lotsize.str.upper()
display(hf_data.hf_lotsize.value_counts(dropna=False)[:20])

Столбец содержит данные о пощади. Возможно, площадь участка.
Около 96000 значений типа NONE, ' ', '—', 'NO DATA', '-- SQFT LOT'
Есть измерения в квадратных футах и акрах.
Переведем акры в квадратные футы

In [ ]:
def parse_lotsize(s):
    s = str(s)
    if len(s) == 0 or s == ' ':  return '', 0.0
    elif 'nan' in s: return '', 0.0
    elif s == '' or 'NO' in s or '—' in s or '-' in s: return '', 0.0
 
    elif ' SQ' in s: 
        s = s.replace(',', '')
        s = s.replace('.', '')
        s = s.replace('"', '')
        s = s.replace("'", '')
        s = s.split(' ')
        try:
            return 'SQ', float(s[0])
        except:
            # print('EX_SQ', s)
            return 'SQ_EX', 0.0

    elif ' AC' in s: 
        s = s.replace(',', '')
        s = s.replace('.', '')
        s = s.replace('"', '')
        s = s.replace("'", '')
        s = s.split(' ')
        try:
            return 'AC', float(s[0])*43560 # переводим в квадратные футы
        except:
            # print ('EX_AC', s)
            return 'AC_EX', 0.0
    
    else: 
        # print ('EX', s)
        return 'EX', s

In [ ]:
hf_lotsize = hf_data.hf_lotsize.map(parse_lotsize).astype('category')
display(hf_lotsize.value_counts(dropna=False))

In [ ]:
idx, values = zip(*hf_lotsize)
a = pd.Series(values, idx)
display(a.index.value_counts(dropna=False))

Данные гразные. Площадь есть в акрах, есть в квадратных футах, а есть без указания единиц измерения. Такой столбец очень, проблематично чистить. 
Тем более он будет сильно коллерировать с столбцом "sqft" в основном датасете

In [ ]:
# Удалим hf_lotsize
delete_col.append('hf_lotsize')

In [ ]:
# hf_price_sqft
print('Пропуски:', hf_data.hf_price_sqft.isna().sum())
print('Уникальных:', hf_data.hf_price_sqft.nunique())
hf_data.hf_price_sqft = hf_data.hf_price_sqft.str.upper()
display(hf_data.hf_price_sqft.value_counts(dropna=False))

В этом столбце представлена площадь за квадратный метр. 
65627 нулевых значений. Можно попробовать заменить их, например, медианными значениями по городу.

In [ ]:
# TODO Пока удалим но нужно подумать
# Удалим hf_lotsize
delete_col.append('hf_price_sqft')

In [ ]:
display(data.head(2))
display(hf_data.head(2))

In [ ]:
display(data.info())
display(hf_data.info())

In [ ]:
# Объединим с итоговым датасетом
delete_col.append('homeFacts')
data_new = data.merge(hf_data, left_index=True, right_index=True)
data_new.head(2).T

### Column "fireplace"

In [ ]:
# fireplace
print('Пропуски:', data_new.fireplace.isna().sum())
print('Уникальных:', data_new.fireplace.nunique())
data_new.fireplace = data_new.fireplace.str.lower()
display(data_new.fireplace.value_counts(dropna=False))

In [ ]:
def clen_fireplace(string):
  if type(string)!=str: return 0
  if(len(str(string))==0): return 0
  res = 0
  res = re.findall(r'\d+', str(string)) 
  if(res): return int(res[0])
  if('yes' in string): return 1
  if('not applicable' in string): return 0
  if('storage' in string): return 0
  if('one' in string): return 1
  if('two' in string): return 2
  if('three' in string): return 3
  if('four' in string): return 4
  if('five' in string): return 5
  if('six' in string): return 6
  if('seven' in string): return 7
  if('eight' in string): return 8
  if('nine' in string): return 9
  if('ten' in string): return 0
  if('eleven' in string): return 11
  if('twelve' in string): return 12
  return 1

In [ ]:
fireplace = data_new.fireplace.map(clen_fireplace) #.astype('category')
display(fireplace.value_counts(dropna=False))
# data_new.fireplace = fireplace

get_emission_limits(fireplace)

In [ ]:
fireplace.loc[fireplace > 12] = 12
display(fireplace.value_counts(dropna=False))
data_new.fireplace = fireplace

### Column "сity"

In [ ]:
# city
print('Пропуски:', data_new.city.isna().sum())
print('Уникальных:', data_new.city.nunique())
data_new.city = data_new.city.str.upper()
display(data_new.city.value_counts(dropna=False))

Удалим 34 записи с пропусками городов.
Есть идея, подключить внешние данные с рейтингами городов по нескольким параметрам:
* криминальный рейтинг города
* экологический рейтинго города
* деловой рейтинг города
* образовательный рейтинг города
* общий рейтинг города у релтеров

In [ ]:
data_new.dropna(subset=['city'], inplace=True)
data_new.city = data_new.city.astype('category')
print('Пропуски:', data_new.city.isna().sum())

### Column "schools"

In [ ]:
print('Пропуски:', data_new.schools.isna().sum())
print('Уникальных:', data_new.schools.nunique())
display(data_new.schools[0])
# data_new.info()

In [ ]:
def parse_school(s):
   s = s[1:-1]
   s = s.replace("'", '"')
   s = s.split(', "name"')
  #  pprint(s)
  #  pprint(s[0])
   s = s[0] + "}"
   s = s.replace(' None, ', ' "None", ')

   #print(ind, s)
   d = json.loads(s)
   dictance = []
   grades = []
   rating = []
   grades = d['data']['Grades']
   for i in d['data']['Distance']:
     i = float(i.replace('mi', ''))
     dictance.append(i)
   for i in d['rating']:
     if '/'in i:
       i = i.split('/')[0]
     rating.append(i)

   return dictance, grades, rating

In [ ]:
schools = data_new.schools.map(parse_school) #.astype('category')
# schools.columns = ['sk_dist', 'sk_grades', 'sk_rating']
schools[0]

In [ ]:
school_data = pd.DataFrame.from_records(schools, columns = ['sk_dist', 'sk_grades', 'sk_rating'])
display(school_data.head(5))
display(school_data.info())

In [ ]:
def school_distance_min_and_meam(dist):
   dist = list(dist)
   if len(dist) == 0:
     min = mean = 0
   else:
     d = np.array(dist)
     min = d.min()
     mean = d.mean()
   
   return np.round(min, 1), np.round(mean, 1)

In [ ]:
# Создадим признак минимального и среднего расстояния до школы
x = school_data.sk_dist.map(school_distance_min_and_meam) #.astype('category')
new_features = pd.DataFrame.from_records(x, columns = ['sk_distance_min', 'sk_distance_mean'])

school_data  = school_data.merge(new_features, left_index=True, right_index=True)
school_data.head(2).T

In [ ]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def school_rating(r):
   a = []
   if len(r) == 0:
     min = max = mean = 0
   else:
     for i in r:
       #print(type(i))
       if is_number(i):
         a.append(int(i))
   if len(a) == 0:
     min = max = mean = 0
   else:  
     d = np.array(a)
     min = d.min()
     max = d.max()
     mean = d.mean()
   
   return np.round(min,1), np.round(max, 1), np.round(mean, 1)

In [ ]:
x = school_data.sk_rating.map(school_rating)
new_features = pd.DataFrame.from_records(x, columns = ['sk_rating_min', 'sk_rating_max', 'sk_rating_mean'])

school_data  = school_data.merge(new_features, left_index=True, right_index=True)
school_data.head(2).T

In [ ]:
def grade_replace(s):
  g = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
  s = s.replace('PK', '0')
  s = s.replace('Pk', '0')
  s = s.replace('K', '0')
  s = s.replace('Preschool to ', '0-')
  s = s.replace('N/A', '999')
  s = s.replace('NA', '999')
  s = s.replace('None', '999')
  s = s.replace(' - ', '-')
  s = s.replace(' – ', '-')
  s = s.replace('–', '-')
  s = s.replace(' to ', '-')
  s = s.split(', ')
  
  if s[0] == '999':
    return g
  
  a = []
  a = s[0].split('-')
  if len(a) == 2:
    for i in range(int(a[0]), int(a[1])+1):
      g[i] = g[i] + 1
  else:
    g[int(a[0])] = g[int(a[0])] + 1

  if len(s) == 2:
    b = []
    b = s[1].split('-')
    if len(b) == 2:
      for i in range(int(b[0]), int(b[1])+1):
        g[i] = g[i] + 1
      else:
        g[int(b[0])] = g[int(b[0])] + 1

  return g

def sk_g(l):
   g = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
   if type(l) != list:
     return g

   if len(l) == 0:
     return g

   for i in l:
     g = g + grade_replace(i)

   return g

In [ ]:
x = school_data.sk_grades.map(sk_g)
new_features = pd.DataFrame.from_records(x, columns = ['sk_gr_pk','sk_gr_01', 'sk_gr_02', 'sk_gr_03', 'sk_gr_04', 'sk_gr_05', 'sk_gr_06', 'sk_gr_07', 'sk_gr_08', 'sk_gr_09', 'sk_gr_10', 'sk_gr_11', 'sk_gr_12'])

school_data  = school_data.merge(new_features, left_index=True, right_index=True)
school_data.head(2).T

В результате получили следующие столбцы:
* **sk_dist_min** - минимальное расстояние до школы
* **sk_dist_mean** - среднее расстояние до школы
* **sk_rating_min** - минимальный рейтинг школы
* **sk_rating_max** - максимальный рейтинг школы
* **sk_rating_mean** - средний рейтинг школ в окрестностях
* **sk_gp_pk** - количество детских садов (дощкольных образовательных учреждений) в окресностях
* **sk_gr_01** по **sk_gr_12** - количество школ с соответсвующими классами обучения (с 1 по 12) в окрестностях

In [ ]:
school_data.drop(['sk_dist', 'sk_grades', 'sk_rating'], axis=1, inplace=True)
school_data.info()

In [ ]:
display(data_new.info())
display(school_data.info())

In [ ]:
# Удалим признак schools и объедим с новым датасетом
delete_col.append('schools')
data_new  = data_new.merge(school_data, left_index=True, right_index=True)
data_new.head(2).T

### Column "sqft"

In [ ]:
print('Пропуски:', data_new.sqft.isna().sum())
print('Уникальных:', data_new.sqft.nunique())
data_new.sqft = data_new.sqft.str.upper()
display(data_new.sqft.value_counts(dropna=False))

In [ ]:
sqft = data_new.sqft.replace(r'\D+',  np.nan,  regex=True)
display(sqft.value_counts(dropna=False))

In [ ]:
sqft.astype(float)

In [ ]:
# sqft.
def clen_nan(string):
  if type(string)!=str: 
      print(string)

# sqft.map(clen_nan)
# TODO проблемный признак пока оставим

# Удалим
delete_col.append('sqft')

### Column "zipcode"

In [ ]:
print('Пропуски:', data_new.zipcode.isna().sum())
print('Уникальных:', data_new.zipcode.nunique())
data_new.zipcode = data_new.zipcode.str.upper()
display(data_new.zipcode.value_counts(dropna=False))

In [ ]:
# TOTO Пока удалим
delete_col.append('zipcode')

### Column "beds"

In [ ]:
print('Пропуски:', data_new.beds.isna().sum())
print('Уникальных:', data_new.beds.nunique())
data_new.beds = data_new.beds.str.upper()
display(data_new.beds.value_counts(dropna=False))

In [ ]:
def parse_beds(s):
    s = str(s)
    if len(s) == 0 or s == ' ':  return 0, 0
    if 'nan' in s: return 0, 0
    if '--' in s: return 0, 0
    if 'BAT' in s: return 1, 0

    if ' BED' in s or ' BD' in s: 
        s = s.split(' ')
        try:
            return float(s[0]), 0
        except:
            return 1, 0

    if ' SQ' in s: 
        s = s.split(' ')
        try:
            return 1, float(s[0])
        except:
            # print('EX_SQ', s)
            return 1, 0

    if ' AC' in s: 
        s = s.split(' ')
        try:
            return 1, float(s[0])*43560
        except:
            # print ('EX_AC', s)
            return 1, 0
    
    try:
        n = s.split(' ')
        try:
            #print ("EX1", s)
            return float(n[0]), 0
        except:
            # print ('EX2', s)
            return 1, 0
    except:
         return 1, 0
    #     try:
    #         print ("EX3", s)
    #         return float(s), 0
    #     except:
    #         print ('EX4', s)
    #         return 1, 0

In [ ]:
beds = data_new.beds.map(parse_beds) #.astype('category')
display(beds.value_counts(dropna=False))

data_beds = pd.DataFrame(beds.tolist(), columns=['beds_num', 'beds_area'])
display(data_beds.head(5))
data_beds.info()

In [ ]:
display(data_new.info())
display(data_beds.info())

In [ ]:
display(data_beds.astype(int).value_counts(dropna=False))
data_beds = data_beds.astype(int)
print('Пропуски:', data_beds.isna().sum())

In [ ]:
# Удалим признак beds
delete_col.append('beds')
data_new = data_new.join(data_beds)
display(data_new.head(2))
data_new.info()

### Column "state"

In [ ]:
print('Пропуски:', data_new.state.isna().sum())
print('Уникальных:', data_new.state.nunique())
data_new.state = data_new.state.str.upper()
display(data_new.state.value_counts(dropna=False)[:25])
# Всего 38 уникальных штата.
# Большинство обявлений о продаже в Филадельфии (FL) и Техасе (TX).
data_new.state = data_new.state.astype('category')

### Column "stories" 

In [ ]:
print('Пропуски:', data_new.stories.isna().sum())
print('Уникальных:', data_new.stories.nunique())
data_new.stories = data_new.stories.str.lower()
display(data_new.stories.value_counts(dropna=False))

In [ ]:
def parse_stories(string):
  if type(string)!=str: return 0 # Пропуски 150103
  if(len(str(string))==0): return 0
  string.replace(',', '.')
  res = []
  res = re.findall(r'\d+\.\d+', str(string)) 
  if(res): return float(res[0])
  res = re.findall(r'\d+', str(string)) 
  if(res): return float(res[0])
  if('yes' in string): return 1
  if('not applicable' in string): return 0
  if('storage' in string): return 0
  if('one' in string): return 1
  if('two' in string): return 2
  if('three' in string): return 3
  if('four' in string): return 4
  if('five' in string): return 5
  if('six' in string): return 6
  if('seven' in string): return 7
  if('eight' in string): return 8
  if('nine' in string): return 9
  if('ten' in string): return 0
  if('eleven' in string): return 11
  if('twelve' in string): return 12
  return 0 # если не распарсили то пусть будет 0

In [ ]:
stories = data_new.stories.map(parse_stories) #.astype('category')
display(stories.value_counts(dropna=False))
perc1, min_emission_limits, max_emission_limits, perc99 = get_emission_limits(stories)

In [ ]:
# data_new.stories.loc[data_new.stories < min_emission_limits] = 0 #int(perc1)
stories.loc[stories > max_emission_limits] = int(perc99)
display(stories.value_counts(dropna=False))
#data_new.stories = stories.astype(int)
data_new.stories = stories

### Column "target"

In [ ]:
print('Пропуски:', data_new.target.isna().sum())
print('Уникальных:', data_new.target.nunique())

In [ ]:
# Удалим 2476 пустых значения
data_new.dropna(subset=['target'], inplace=True)

print('Пропуски:', data_new.target.isna().sum())
print('Уникальных:', data_new.target.nunique())

data_new.target = data_new.target.str.upper()
display(data_new.target.value_counts(dropna=False))

In [ ]:
target = data_new.target.apply(lambda x: x if type(x)!=str else x
                                .replace('$', '')
                                .replace('+', '')
                                .replace(',', '')
                                ).astype(int)

# display(data.target.value_counts(dropna=False))
data_new.target = target
# data_new['target_log'] = np.log(data_new.target)

In [ ]:
# data_new.target.plot()
data_new.target.plot(figsize=(12,5));
plt.tight_layout()

### Результаты EDA
* Обработал данные, создал новые признаки, прологарифмировал целевую переменную
* Из-за большого количества пропусков в данных приходилось менять их на значения "NO_DATA" 
* Если останется время, можно получить дополнительную информацию о городах из внешних источников.

In [ ]:
display(data_new.head(3))
data_new.info()

In [ ]:
delete_col

In [ ]:
dataset =  data_new.drop(delete_col, axis=1)
dataset.status = dataset.status.astype('category')
dataset.info()

In [ ]:
dataset.dropna(inplace=True)
dataset.info()

### Код ML вынесем в отдельный блокнот